In [5]:
import os
import json
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings
from jinja2 import Template

pdf_path = "INFO1 grundlagen.pdf"
loader = PyPDFLoader(pdf_path)
document = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = splitter.split_documents(document)

embeddings = OllamaEmbeddings(model="gemma:7b")
db = FAISS.from_documents(docs, embeddings)
retriever = db.as_retriever()

# LLM-Setup
llm = OllamaLLM(
    base_url="http://134.96.217.20:53100",
    model="gemma:7b",
    temperature=0.7,
    top_p=0.9,
    top_k=20,
    repeat_penalty=1.2,
    presence_penalty=1.5
)
prompt = PromptTemplate(
    input_variables=["frage"],
    template=(
        "Du bist ein freundlicher Assistent, der Kindern auf ihre Fragen antwortet.\n"
        "Erkläre Dinge in einfacher, kindgerechter Sprache, damit Kinder im Alter von 6 bis 10 Jahren sie gut verstehen können.\n"
        "Sei positiv, ermutigend und benutze kurze, klare Sätze.\n"
        "Wenn es passt, darfst du Beispiele oder kleine Vergleiche verwenden, damit es noch besser verständlich wird.\n\n"
        "Frage: {frage}\n"
        "Antwort:"
    )
)

qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type_kwargs={"prompt": prompt})

html_template_str = """
<h1>Chatbot-Antwort aus Dokument</h1>
<p><strong>Frage:</strong> {{ frage }}</p>
<p><strong>Antwort:</strong> {{ antwort }}</p>
<p><strong>Beispiel:</strong> {{ beispiel }}</p>
"""

print("🤖 Chatbot gestartet! Tipp 'exit' oder 'quit' zum Beenden.\n")

# Chat-Schleife
while True:
    user_input = input("Du: ")
    if user_input.lower() in ["exit", "quit"]:
        print("🤖 Bot: Auf Wiedersehen!")
        break
try:
    response_text = qa_chain.run(user_input)

    try:
        response = json.loads(response_text)
        print("🤖 Bot:")
        print(json.dumps(response, indent=2, ensure_ascii=False))

        html_template = Template(html_template_str)
        html_content = html_template.render(
            frage=user_input,
            antwort=response.get('answer', 'Keine Antwort gefunden.'),
            beispiel=response.get('example', 'Kein Beispiel verfügbar.')
        )
        safe_filename = user_input[:30].replace(' ', '_').replace('?', '').replace('/', '')
        pdf_filename = f"antwort_{safe_filename}.pdf"
        #HTML(string=html_content).write_pdf(pdf_filename)
        print(f"📄 PDF gespeichert als: {pdf_filename}")

    except json.JSONDecodeError:
        print("⚠️ Die Antwort war kein gültiges JSON:")
        print(response_text)

except Exception as e:
    print(f"⚠️ Fehler: {e}")



C:\Users\marim\AppData\Local\Temp\ipykernel_18912\3134179662.py:19: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="gemma:7b")


ValueError: Error raised by inference API HTTP code: 500, {"error":"model requires more system memory (9.5 GiB) than is available (8.9 GiB)"}